In [8]:
#Baseline model overview
#Vehicles arrive on a go-straight-only, single lane according to a Poisson process 
#with an average interarrival time equal to ? minutes, 
#while pedestrians from both sides of the road arrive according to a Poisson process 
#with an average interarrival time equal to minutes. 
#Assume pedestrians cross the road only when the traffic light is green( which means red for the vehicles) 
#and clear the intersection as a group according to a triangular distribution 
#with lower bound, mode and upper bound equal to ? minutes, ? minutes, and ? minutes respectively. 
#Pedestrians' positions are not taken into account.
#baseline model was inspired by Dr. Phillip M. Feldman, link can be found here: http://phillipmfeldman.org/Python/discrete_event_simulation/traffic_sim.py

# Import all necessary packages
# double-ended queue
from collections import deque 
import numpy as np
import simpy
from simpy.util import start_delayed

#define a utility class to keep track of the stats
class Struct(object): 
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

random_seed = 0
np.random.seed(random_seed)

end_time= 60

# Cars cars arrive at the traffic light according to a Poisson process 
t_interarrival_mean = 7.91/60

p_interarrival_mean = 30/60 #interarrival time for pedestrian set to be 30 seconds

# Traffic light green and red durations:
t_green= 26/60
t_red= 90/60

# The time for a car at the head of the queue to depart (clear the intersection)
# is modeled as a triangular distribution with specified minimum, maximum, and
# mode.
t_depart_left= 1.8/60
t_depart_mode= 2/60 
t_depart_right= 2.2/60

#the time for a group of pedestrians to cross the road is modeled as a normal distribution
p_crossing_mode = 18/60 
p_crossing_lower = 13/60
p_crossing_upper = 23/60

#initially, no cars are waiting at the light:
queue= deque()
pedestrian = deque()

pedestrian_count = arrival_count= departure_count= 0

Q_stats= Struct(count=0, cars_waiting=0)
W_stats= Struct(count=0, waiting_time=0.0)
P_W_stats = Struct(count=0, waiting_time=0.0)
p_num_stats = Struct(count=0, pedestrian_waiting=0)

def arrival():
    global arrival_count, env, light, queue

    while True:
        
        arrival_count+= 1

        if light == 'red' or len(queue):
            queue.append((arrival_count, env.now))
            print(f"Car {arrival_count} arrived and joined the queue at position {len(queue)} at time {env.now}")

        else:
            print("Car #%d arrived to a green light with no cars waiting at time "
           "%.3f." % (arrival_count, env.now))
            W_stats.count+= 1

      #schedule next arrival:
        yield env.timeout( np.random.exponential(t_interarrival_mean))

def departure():
    global env, queue

    while True:
        
        car_number, t_arrival= queue.popleft()
        print(f"Car {car_number} departed at time {env.now}, leaving {len(queue)} cars in the queue.")

        W_stats.count += 1
        W_stats.waiting_time += env.now - t_arrival

        if light == 'red' or len(queue) == 0:
            return

        delay = np.random.triangular(left=t_depart_left, mode=t_depart_mode,
        right=t_depart_right)

      #schedule next departure:
        yield env.timeout(delay)


def Pedestrian():
    
    global env, light, pedestrian, pedestrian_count
    
    while True:
        
        pedestrian_count += 1
        
        if light == 'red' and len(pedestrian): #assume red light for vehicles means green for pedestrians
            print(f'At time {env.now} pedestrian {pedestrian_count} arrived at the intersection and started to cross the road.')
            P_W_stats.count += 1
        else:
            pedestrian.append((pedestrian_count, env.now))
            print(f'At time {env.now}, pedestrian {pedestrian_count} arrived and started to wait.')
        
        yield env.timeout(np.random.exponential(p_interarrival_mean))
    
def pedestrian_cross():
    global env, pedestrian
    
    while True:
        
        pedestrian_idx, t_arrival = [i[0] for i in pedestrian], [i[1] for i in pedestrian]
        P_W_stats.count += len(pedestrian)
        P_W_stats.waiting_time += len(pedestrian)*env.now - sum(t_arrival)
        pedestrian = deque()
        print(f'Pedestrians {pedestrian_idx} crossed the road at time {env.now}.')
        
        if light == 'green' or len(pedestrian)==0: #assume green light for vehicles means red light for pedestrian
            return
        #schedule next crossing
        yield env.timeout(np.random.triangular(p_crossing_lower,p_crossing_mode,p_crossing_upper))

def light():
    
    global env, light

    while True:
        
        light = 'green'
        print(f"The light turned green at time {env.now}.")

        if len(queue):
            delay= np.random.triangular(left=t_depart_left, mode=t_depart_mode,
            right=t_depart_right)

            start_delayed(env, departure(), delay=delay)

      # Schedule event that will turn the light red:
        yield env.timeout(t_green)

        light= 'red'
        print(f"The light turned red at time {env.now}.")
        
        if len(pedestrian):
            start_delayed(env,pedestrian_cross(),delay=np.random.triangular(p_crossing_lower,p_crossing_mode,p_crossing_upper))

      # Schedule event that will turn the light green:
        yield env.timeout(t_red)


def monitor():
    global env, Q_stats, p_num_stats 
    while True:
        Q_stats.count += 1
        Q_stats.cars_waiting += len(queue)
        p_num_stats.count += 1
        p_num_stats.pedestrian_waiting += len(pedestrian)
        #check the number of vehicles and the number of pedestrians every 1/60 min
        yield env.timeout(1) 

print("\nSimulation of Cars Arriving at Intersection Controlled by a Traffic "
  "Light\n\n")

#initialize environment:
env= simpy.Environment()

#schedule first change of the traffic light:
env.process(light())

#schedule first arrival of a car:
t_first_arrival = np.random.exponential(t_interarrival_mean)
start_delayed(env, arrival(), delay=t_first_arrival)

p_t_first_arrival = np.random.exponential(p_interarrival_mean)
start_delayed(env, Pedestrian(), delay=p_t_first_arrival)

#schedule first statistical monitoring event:
env.process(monitor())

env.run(until=end_time)


#report statistics.

print(f"\n\n      *** Statistics( random seed={random_seed}) ***")
print(' ( Baseline model with no distracted driver)\n\n')
print('Vehicle stats:\n')
print(f"Number of vehicles waiting in queue per minute: {Q_stats.cars_waiting / float(Q_stats.count)}")
print(f"Average waiting time for a vehicle (minutes): {W_stats.waiting_time / float(W_stats.count)}")

print('\n\nPedestrian stats:\n')
print(f"Number of pedestrians waiting to cross per minute: {p_num_stats.pedestrian_waiting / float(p_num_stats.count)}")
print(f"Average waiting time for a pedestrian (minutes): {P_W_stats.waiting_time / float(P_W_stats.count)}")



Simulation of Cars Arriving at Intersection Controlled by a Traffic Light


The light turned green at time 0.
Car #1 arrived to a green light with no cars waiting at time 0.105.
Car #2 arrived to a green light with no cars waiting at time 0.227.
Car #3 arrived to a green light with no cars waiting at time 0.330.
Car #4 arrived to a green light with no cars waiting at time 0.403.
The light turned red at time 0.43333333333333335.
Car 5 arrived and joined the queue at position 1 at time 0.5399242857546558
Car 6 arrived and joined the queue at position 2 at time 0.6157969001783882
At time 0.6279653814829189, pedestrian 1 arrived and started to wait.
Car 7 arrived and joined the queue at position 3 at time 0.9089315352965268
Car 8 arrived and joined the queue at position 4 at time 0.9726864122654739
Car 9 arrived and joined the queue at position 5 at time 1.1795192200514093
Car 10 arrived and joined the queue at position 6 at time 1.2787467568253161
Car 11 arrived and joined the queue at p

In [9]:
# Import all necessary packages
# double-ended queue
from collections import deque 
import numpy as np
import simpy
from simpy.util import start_delayed
from colorama import Fore, Back, Style 

#define a utility class to keep track of the stats
class Struct(object): 
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

random_seed = 0
np.random.seed(random_seed)

end_time= 60

# Cars cars arrive at the traffic light according to a Poisson process 
t_interarrival_mean = 7.91/60

p_interarrival_mean = 30/60 #interarrival time for pedestrian set to be 30 seconds

# Traffic light green and red durations:
t_green= 26/60
t_red= 90/60

# The time for a car at the head of the queue to depart (clear the intersection)
# is modeled as a triangular distribution with specified minimum, maximum, and
# mode.
t_depart_left= 1.8/60
t_depart_mode= 2/60 
t_depart_right= 2.2/60

#the time for a group of pedestrians to cross the road is modeled as a normal distribution
p_crossing_mode = 18/60 
p_crossing_lower = 13/60
p_crossing_upper = 23/60

#initially, no cars are waiting at the light:
queue= deque()
pedestrian = deque()

pedestrian_count = arrival_count= departure_count= 0

Q_stats= Struct(count=0, cars_waiting=0)
W_stats= Struct(count=0, waiting_time=0.0)
P_W_stats = Struct(count=0, waiting_time=0.0)
p_num_stats = Struct(count=0, pedestrian_waiting=0)

distracted_p = 0.2

def arrival():
    global arrival_count, env, light, queue, distracted_boolean
    distracted_boolean = np.random.choice([True,False],p=[distracted_p,1-distracted_p])

    while True:
        
        arrival_count+= 1

        if light == 'red' or len(queue):
            queue.append((arrival_count, env.now))
            print(f"Car {arrival_count} arrived and joined the queue at position {len(queue)} at time {env.now}")

        else:
            print("Car #%d arrived to a green light with no cars waiting at time "
           "%.3f." % (arrival_count, env.now))
            W_stats.count+= 1

      #schedule next arrival:
        if distracted_boolean:
            print(Fore.YELLOW + 'Watch out! Distracted driver!')
            print(Style.RESET_ALL)
            yield env.timeout( np.random.exponential(t_interarrival_mean) + 1)
        else:
            yield env.timeout( np.random.exponential(t_interarrival_mean))

def departure():
    global env, queue, distracted_boolean
    distracted_boolean = np.random.choice([True,False],p=[distracted_p,1-distracted_p])

    while True:
        
        car_number, t_arrival= queue.popleft()
        print(f"Car {car_number} departed at time {env.now}, leaving {len(queue)} cars in the queue.")

        W_stats.count += 1
        W_stats.waiting_time += env.now - t_arrival

        if light == 'red' or len(queue) == 0:
            return

        delay = np.random.triangular(left=t_depart_left, mode=t_depart_mode,
        right=t_depart_right)

      #schedule next departure:
        if distracted_boolean:
            print(Fore.YELLOW + 'Watch out! Distracted driver!')
            print(Style.RESET_ALL)
            yield env.timeout(delay + 1)
        else:
            yield env.timeout(delay) 


def Pedestrian():
    
    global env, light, pedestrian, pedestrian_count
    
    while True:
        
        pedestrian_count += 1
        
        if light == 'red' and len(pedestrian): #assume red light for vehicles means green for pedestrians
            print(f'At time {env.now} pedestrian {pedestrian_count} arrived at the intersection and started to cross the road.')
            P_W_stats.count += 1
        else:
            pedestrian.append((pedestrian_count, env.now))
            print(f'At time {env.now}, pedestrian {pedestrian_count} arrived and started to wait.')
        
        yield env.timeout(np.random.exponential(p_interarrival_mean))
    
def pedestrian_cross():
    global env, pedestrian
    
    while True:
        
        pedestrian_idx, t_arrival = [i[0] for i in pedestrian], [i[1] for i in pedestrian]
        P_W_stats.count += len(pedestrian)
        P_W_stats.waiting_time += len(pedestrian)*env.now - sum(t_arrival)
        pedestrian = deque()
        print(f'Pedestrians {pedestrian_idx} crossed the road at time {env.now}.')
        
        if light == 'green' or len(pedestrian)==0: #assume green light for vehicles means red light for pedestrian
            return
        #schedule next crossing
        yield env.timeout(np.random.triangular(p_crossing_lower,p_crossing_mode,p_crossing_upper))

def light():
    
    global env, light

    while True:
        
        light = 'green'
        print(f"The light turned green at time {env.now}.")

        if len(queue):
            delay= np.random.triangular(left=t_depart_left, mode=t_depart_mode,
            right=t_depart_right)
            if distracted_boolean:
                start_delayed(env, departure(), delay=delay + 1)
            else:
                start_delayed(env, departure(), delay=delay)

      # Schedule event that will turn the light red:
        yield env.timeout(t_green)

        light= 'red'
        print(f"The light turned red at time {env.now}.")
        
        if len(pedestrian):
            start_delayed(env,pedestrian_cross(),delay=np.random.triangular(p_crossing_lower,p_crossing_mode,p_crossing_upper))

      # Schedule event that will turn the light green:
        yield env.timeout(t_red)


def monitor():
    global env, Q_stats, p_num_stats 
    while True:
        Q_stats.count += 1
        Q_stats.cars_waiting += len(queue)
        p_num_stats.count += 1
        p_num_stats.pedestrian_waiting += len(pedestrian)
        #check the number of vehicles and the number of pedestrians every 1/60 min
        yield env.timeout(1) 

print("\nSimulation of Cars Arriving at Intersection Controlled by a Traffic "
  "Light\n\n")

#initialize environment:
env= simpy.Environment()

#schedule first change of the traffic light:
env.process(light())

#schedule first arrival of a car:
t_first_arrival = np.random.exponential(t_interarrival_mean)
start_delayed(env, arrival(), delay=t_first_arrival)

p_t_first_arrival = np.random.exponential(p_interarrival_mean)
start_delayed(env, Pedestrian(), delay=p_t_first_arrival)

#schedule first statistical monitoring event:
env.process(monitor())

env.run(until=end_time)


#report statistics.

print(f"\n\n      *** Statistics( random seed={random_seed}) ***")
print(f' ( Baseline model with {distracted_p*100}% distracted drivers)\n\n')
print('Vehicle stats:\n')
print(f"Number of vehicles waiting in queue per minute: {Q_stats.cars_waiting / float(Q_stats.count)}")
print(f"Average waiting time for a vehicle (minutes): {W_stats.waiting_time / float(W_stats.count)}")

print('\n\nPedestrian stats:\n')
print(f"Number of pedestrians waiting to cross per minute: {p_num_stats.pedestrian_waiting / float(p_num_stats.count)}")
print(f"Average waiting time for a pedestrian (minutes): {P_W_stats.waiting_time / float(P_W_stats.count)}")




Simulation of Cars Arriving at Intersection Controlled by a Traffic Light


The light turned green at time 0.
Car #1 arrived to a green light with no cars waiting at time 0.105.
Car #2 arrived to a green light with no cars waiting at time 0.209.
Car #3 arrived to a green light with no cars waiting at time 0.281.
Car #4 arrived to a green light with no cars waiting at time 0.418.
The light turned red at time 0.43333333333333335.
Car 5 arrived and joined the queue at position 1 at time 0.4940853154565517
At time 0.6279653814829189, pedestrian 1 arrived and started to wait.
Car 6 arrived and joined the queue at position 2 at time 0.7872199505746903
Car 7 arrived and joined the queue at position 3 at time 0.8509748275436374
Car 8 arrived and joined the queue at position 4 at time 1.0578076353295727
Car 9 arrived and joined the queue at position 5 at time 1.1570351721034795
Car 10 arrived and joined the queue at position 6 at time 1.2677004023985476
Car 11 arrived and joined the queue at p